<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2022-1/blob/main/5_Algoritmos_greedy_y_recursion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En esta sesión veremos problemas que involucren ideas recursiva, así como greedy. Con la finalidad de reforzar ambos temas dentro de la resolución de problemas.

**Ejemplo 1.** Dada una lista $L$ de $n$ números enteros, a cada número de la lista se le puede aplicar la siguiente operación tantas veces como uno quiera: cambiar el valor $L[i]$ por $ - L[i] - 1$. Determina una nueva lista $M$ tal que se pueda llegar de $L$ a $M$ con operaciones de este estilo, y que el producto de sus elementos sea lo mayor posible.

Una primera observación para el problema es que $-(-x-1) - 1 = x$, es decir, si aplicamos dos veces una operación a un mismo elemento volvemos al elemento inicial. Por lo que a cada elemento de la lista le aplicamos a lo más una operación. Un posible camino sería contestar un problema un poco más general, consiguiendo una lista $M_L$ que maximice el producto (que será no negativo) y otra $N_L$ que lo minimice (que tendrá producto negativo). Tenemos dos opciones para $M[i]$, que sea $L[i]$ o que sea $-L[1]-1$, notemos que si $L[i]$ es no negativo, entonces $-L[i]-1$ es negativo, y que si $L[i]$ es negativo, entonces $ - L[i] - 1$ es no negativo. Con esto en mente, si $P$ es la lista que consta de los primeros $n-1$ elementos de $L$, se tiene la siguiente recursión:

*   `M_L = max(M_P + max(L[i], -L[i]-1), N_P + min(L[i], -L[i] - 1))`
*   `N_L = min(M_P + min(L[i], -L[i]-1), N_P + max(L[i], -L[i]-1))`

Veamos una implementación de este algoritmo.



In [ ]:
def prod(L):  # Nos regresa N, M y el producto de sus elementos
  n = len(L)
  if(len(L) == 1):
    if(L[0] >= 0):
      return L, L[0], [-L[0]-1], -L[0]-1
    else:
      return [-L[0]-1], -L[0]-1, L, L[0]
  M_p, aux1, N_p, aux2 = prod(L[:n-1])
  M = []
  prodm = 0
  N = []
  prodn = 0
  if(L[n-1] >= 0):
    if(aux1*L[n-1] > aux2*(-L[n-1]-1)):
      M = M_p + [L[n-1]]
      prodm = aux1*L[n-1]
    else:
      M = N_p + [-L[n-1]-1]
      prodm = aux2*(-L[n-1]-1)
    if(aux1*(-L[n-1]-1) < aux2*L[n-1]):
      N = M_p + [-L[n-1] - 1]
      prodn = aux1*(-L[n-1]-1)
    else:
      N = N_p + [L[n-1]]
      prodn = aux2*L[n-1]
  else:
    if(aux2*L[n-1] > aux1*(-L[n-1]-1)):
      M = N_p + [L[n-1]]
      prodm = aux2*L[n-1]
    else:
      M = M_p + [-L[n-1]-1]
      prodm = aux1*(-L[n-1]-1)
    if(aux2*(-L[n-1]-1) < aux1*L[n-1]):
      N = N_p + [-L[n-1] - 1]
      prodn = aux2*(-L[n-1]-1)
    else:
      N = M_p + [L[n-1]]
      prodn = aux1*L[n-1]
  return M, prodm, N, prodn



L = [-3,-3,2]
print(prod(L))


([2, -3, -3], 18, [-3, -3, -3], -27)


¿Cuál es la complejidad del algoritmo anterior? ¿Se puede mejorar?

Después de checar algunos casos, uno se puede convencer de que el arreglo final que tiene producto máximo cumple que todos sus elementos son negativos excepto a lo más uno de ellos. ¿Por qué pasa esto? Consideremos primero el problema de maximizar el valor absoluto del producto, dado que $|-x-1| > |x|$ si y solo si $x$ es no negativo, concluimos que la forma en la que se maximiza el valor abosluto del producto es considerando todos los números negativos posibles. Si se tiene que este producto es positivo, hemos terminado, pues será el mayor producto posible, en caso de ser negativo, notemos que es suficiente con aplicar una operación al elemento de menor valor absoluto.

Esto nos da un nuevo algoritmo, veamos su implementación.

In [ ]:
def max_prod(L):
  min = 0
  idx = -1
  for i in range(0, len(L)):
    if(L[i] >= 0):
      L[i] = -L[i]-1
    if(min > L[i]):
      idx = i
      min = L[i]
  if(len(L)%2):
    L[idx] = -L[idx]-1
    return L
  else:
    return L

L = [2,2,2,2,2,2]
print(max_prod(L))

[-3, -3, -3, -3, -3, -3]


**Ejemplo 2.** Consideremos la siguiente variante del problema de las torres de Hanoi. Se tienen $n$ postes (numerados del $1$ al $n$), y en cada poste se encuentra un disco, de radio $a_i$ (todos los radios son distintos), una operación consiste en seleccionar un disco tal que sea el único disco en el poste en ese momento, y moverlo a la cima de alguno de sus postes adyacentes, con la condición de que el disco que quede debajo tenga radio mayor. Determina si es posible hacer que todos los discos se encuentren en un mismo poste bajo una serie de operaciones.

¿Podemos resolver este problema de manera recursiva? Notemos que en caso de ser posible, el disco que queda en la cima del poste es el disco en la primera posición o el disco en la última posición, y además es el de menor radio, con esto en mente, podemos definir una función $f(x,y)$ que nos diga si es posible apilar todos los discos en las posiciones $x, x+1, \dots, y$ en un poste. Se tiene entonces la siguiente recursión: 

*   $f(x,y) = f(x, y-1)$ si $a_y$ es el menor de $a_x, a_{x+1}, \dots, a_y$. 
*   $f(x,y) = f(x+1, y)$ si $a_x$ es el menor de $a_x, a_{x+1}, \dots, a_y$. 

Veamos una implementación de este algoritmo.


In [ ]:
L = [2, 4, 6, 8, 10, 9, 5, 3, 7, 1]

def f(a,b):
  if(a == b):
    return True
  if(L[a] == min(L[a:b+1])):
    return f(a+1, b)
  if(L[b] == min(L[a:b+1])):
    return f(a, b-1)
  return False

print(f(0, len(L)-1))

L = [1,3,5,7,9,8,6,4,2]
print(f(0,len(L)-1))


False
True


¿Podemos hacer un algoritmo greedy para resolver este problema? Notemos que el elemento más grande de la lista no se puede mover de lugar, ¿qué pasa si en cada paso movemos el segundo más grande de la lista que no esté en el poste donde se encuentra el disco más grande? Formalicemos esta idea. Notemos que dada la lista de radios de discos, el objetivo se puede cumplir si y solo si la primer parte está en orden creciente y la segunda está en orden decreciente.

Veamos una implementación de este algoritmo.

In [ ]:
def ans(L):
  if(len(L) == 1):
    return True
  idx = 1
  while(idx < len(L)):
    if(L[idx] < L[idx - 1]):
      break;
    else:
      idx += 1
  idx += 1
  while(idx < len(L)):
    if(L[idx] > L[idx - 1]):
      break;
    else:
      idx += 1
  if(idx == len(L)):
    return True
  else:
    return False

print(ans([2, 4, 6, 8, 10, 9, 5, 3, 7, 1]))
print(ans([1,3,5,7,9,8,6,4,2]))

False
True


Notemos además que el algoritmo greedy nos permite identificar de una manera sencilla las operaciones a seguir para cumplir el objetivo.

**Ejemplo 3.** Supongamos que se tienen $n$ sapos venenosos colocados en una fila, cada uno a distancia de un metro del siguiente. Cada sapo tiene un veneno en la saliva tal que al tocar cualquier ser vivo con la lengua, este morirá un segundo después. Dadas las longitudes de las lenguas de los sapos (en una lista $L$), determina la cantidad de sapos que quedarán vivos si en un momento determinado todos los sapos extienden su lengua al mismo tiempo. 

¿Podemos resolver este problema de manera recursiva? Sí. Notemos que el último sapo sobrevive si y solo si $L[i-1] + (i - 1) < n - i$. Podemos entonces definir una función $g(a,b)$ que nos dice cuántos sapos sobreviven si únicamente consideramos a los que están en posiciones $a, a+1, \dots, b$. Se tiene entonces la siguiente recursión:


*   $g(a,b) = g(a,b-1) + 1$ si se cumple que $L[i-1] + (i-1) < b - i$ para todo $i \in [1, b-1]$. 
*   $g(a,b) = g(a,b-1)$ en caso contrario.

Veamos una implementación de este algoritmo.

In [ ]:
L = [3,0,1,2,0,0,0,3,1,1]

def g(a,b):
  if(a == b):
    return 1
  for i in range(a, b):
    if(L[i-1] + (i-1) >= b-1):
      return g(a,b-1)
  return g(a,b-1) + 1

print(g(1,len(L)))

3


¿Qué pasa si intentamos un algoritmo tipo greedy? Tal vez una idea para un algoritmo greedy sea comenzar a iterar sobre los elementos de la lista, y en cada paso saltar hasta el primer sapo que no muere por causa del sapo que estábamos visitando. Esta idea se puede pulir un poco para llegar al siguiente algoritmo.

Iteramos sobre los elementos de la lista, y vamos guardando un máximo, que nos idica el sapo más a la derecha que sabemos que no sobrevivirá. Si en algún momento llegamos a que dicho máximo es menor que la posición que estamos visitando es porque dicho sapo sobrevivirá, y lo sumamos al conteo final.

Veamos una implementación de este algoritmo.

In [ ]:
def surv(L):
  maxi = -1
  cnt = 0
  for i in range(0, len(L)):
    if(maxi < i):
      cnt += 1
    maxi = max(maxi, i + L[i])
  return cnt

print(surv(L))

3


Los ejemplos vistos se basan en los siguientes problemas:

https://codeforces.com/contest/1180/problem/B

https://codeforces.com/contest/1197/problem/B

https://codeforces.com/contest/892/problem/B

Si quieren practicar más este tipo de problemas, pueden checar en la sección de 'greedy' en la siguiente página:
https://codeforces.com/blog/entry/55274


**Ejercicios.** 

1.   Dada una lista de $n$ números reales $L$, describe e implementa un algoritmo cuya complejidad en tiempo sea menor que $O(n^2)$ tal que nos devuelva un real $x$ que minimice la suma $|L[0] - x| + |L[1] - x| + \dots + |L[n-1] - x|$.
2.   Un robot se encuentra parado en el número $0$ en el intervalo $[0,n]$. Cada segundo el intervalo se llena de lava (que lo derrite por completo), excepto por dos puntos con valores enteros $a_i, b_i$ (estando en el segundo $i \geq 1$). Después de $m$ segundos, el robot sigue sin ser derretido, queremos determinar la menor cantidad de distancia recorrida por el robot para lograr esto. ¿Es cierto que el agoritmo greedy que en cada segundo se mueve al punto libre de lava más cercano cumple con minimizar dicho recorrido? En caso de que tu respuesta sea afirmativa, demuéstralo e implementa el algoritmo respectivo, y en caso de tener una respuesta negativa muestra un contra ejemplo.
3.   Describe e implementa un algoritmo recursivo que permita calcular la menor distancia recorrida por el robot del ejercicio anterior.

Para los ejercicios 2 y 3, puedes suponer que se tiene una lista de parejas $L$, donde cada pareja $L[i-1]$, nos dice los enteros libres de lava en el segundo $i$. 

Unos casos de prueba para los ejercicios 2 y 3 son los siguientes:

Entrada : $m = 2, n = 5$, $L = [(2,3), (1,5)]$

Salida : $3$ es la distancia mínima que tiene que recorrer.

Entrada: $ m = 4, n = 7$, $L = [(1,2), (1, 4), (1,5), (0, 3)]$

Salida : $2$ es la distancia mínima que tiene que recorrer.


*Ejercicio 1.* Aquí va la descripción de tu algoritmo.

In [ ]:
# Aquí va la implementación del algoritmo descrito anteriormente

*Ejercicio 2.* Describe tu algoritmo en caso de tener una respuesta afirmativa, o da un contra ejemplo en caso de que sea negativa.

In [ ]:
# Aquí va la implementación del algoritmo en caso de tener una respuesta afirmativa

*Ejercicio 3.* Aquí va la descripción de tu algoritmo.

In [ ]:
# Aquí va la implementación del algoritmo descrito anteriormente